In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
from wordcloud import WordCloud
import numpy as np

Импортирую таблицу, которую ранее успешно спарсил

In [2]:
df = pd.read_csv('ТаблицаПреобраз.csv')
df

Срок размещения объявления (дней)          Модель машины  Год выпуска  \
0                                     0          BMW X7 4.4 AT         2024   
1                                     0     BMW 7 серия 3.0 AT         2023   
2                                     0          BMW X7 3.0 AT         2024   
3                                     0          BMW XM 4.4 AT         2023   
4                                     5          BMW X7 3.0 AT         2020   
...                                 ...                    ...          ...   
4989                                  7  BMW 6 серия GT 2.0 AT         2020   
4990                                  4     BMW 3 серия 2.0 AT         2014   
4991                                  7     BMW 4 серия 2.0 AT         2014   
4992                                  7     BMW 3 серия 2.0 AT         2019   
4993                                  7     BMW 5 серия 3.0 AT         2019   

      Пробег, км  Цена машины, ₽   Тип машины  Привод Тип топлива  \
0              1        21900000  внедорожник  полный      бензин   
1             50        17000000        седан  полный      дизель   
2              1        16900000  внедорожник  полный      дизель   
3             15        25490000  внедорожник  полный      гибрид   
4          65000        10190000  внедорожник  полный      дизель   
...          ...             ...          ...     ...         ...   
4989       83252         5399000      лифтбек  полный      дизель   
4990      139000         2155000        седан  полный      дизель   
4991       83549         2919700         купе  полный      дизель   
4992       44260         3877140        седан  задний      бензин   
4993       95405         4650000        седан  полный      дизель   

                                                 Ссылка  
0     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
1     https://www.avito.ru/moskva/avtomobili/bmw_7_s...  
2     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
3     https://www.avito.ru/moskva/avtomobili/bmw_xm_...  
4     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
...                                                 ...  
4989  https://www.avito.ru/moskva/avtomobili/bmw_6_s...  
4990  https://www.avito.ru/moskva/avtomobili/bmw_3_s...  
4991  https://www.avito.ru/moskva/avtomobili/bmw_4_s...  
4992  https://www.avito.ru/moskva/avtomobili/bmw_3_s...  
4993  https://www.avito.ru/moskva/avtomobili/bmw_5_s...  

[4994 rows x 9 columns]

1) Интенсивность использования авто (Износ). Будет считаться так:  $ \frac {Износ_-авто}{Максимальный_-износ_-авто_-в_-выборке} $ . Где Износ_авто будет считаться так: $ {(текущий_-год\  -  \ год_-выпуска_-авто) * пробег_-авто}$ - если пробег не равен 0. И так: $ {(текущий_-год\  -  \ год_-выпуска_-авто)} * 100 $ - если пробег равен 0. Умножение на 100 за год необходимо из-за амортизации машины.

In [3]:
iznoc0 = []
iznoc1 = []
for index, i in df.iterrows():
    if i["Пробег, км"] != 0:
        iznoc0.append((2024 - i["Год выпуска"])*i["Пробег, км"])
    elif i["Пробег, км"] == 0:
        iznoc0.append((2024 - i["Год выпуска"])*100)
        
a = max(iznoc0)

for i in iznoc0:
    iznoc1.append(round(i/a, 5)) #до 5-ти знаков, чтобы не было букв в числах
    
df["Износ"] = iznoc1

df.head()

Срок размещения объявления (дней)       Модель машины  Год выпуска  \
0                                  0       BMW X7 4.4 AT         2024   
1                                  0  BMW 7 серия 3.0 AT         2023   
2                                  0       BMW X7 3.0 AT         2024   
3                                  0       BMW XM 4.4 AT         2023   
4                                  5       BMW X7 3.0 AT         2020   

   Пробег, км  Цена машины, ₽   Тип машины  Привод Тип топлива  \
0           1        21900000  внедорожник  полный      бензин   
1          50        17000000        седан  полный      дизель   
2           1        16900000  внедорожник  полный      дизель   
3          15        25490000  внедорожник  полный      гибрид   
4       65000        10190000  внедорожник  полный      дизель   

                                              Ссылка    Износ  
0  https://www.avito.ru/moskva/avtomobili/bmw_x7_...  0.00000  
1  https://www.avito.ru/moskva/avtomobili/bmw_7_s...  0.00000  
2  https://www.avito.ru/moskva/avtomobili/bmw_x7_...  0.00000  
3  https://www.avito.ru/moskva/avtomobili/bmw_xm_...  0.00000  
4  https://www.avito.ru/moskva/avtomobili/bmw_x7_...  0.01012

2. Для большей наглядности необходимо перевести непрерывную переменную Износ в категориальную. Чтобы информация лнгче усваивалась потребителями.

Если значение износа < квантиля 11% - износa нет\
Если значение износа лежит между квантилем 11% (включительно) и 30 - лёгкий износ\
Если значение износа лежит между квантилем 30% (включительно) и квантилем 65% - средний износ\
Если значение износа лежит между квантилем 65% (включительно) и квантилем 90% - сильный износ\
Если значение износа >= 90% - экстремальный износ

In [4]:
iznoc12 = np.sort(iznoc1)

q0 = np.percentile(iznoc12, 11)
q1 = np.percentile(iznoc12, 30)
q2 = np.percentile(iznoc12, 65)
q3 = np.percentile(iznoc12, 90)

def categorize(iznoc):
    if iznoc < q0:
        return "Отсутствует"
    elif iznoc < q1 and iznoc >= q0:
        return "Легкий"
    elif iznoc < q2 and iznoc >= q1:
        return "Средний"
    elif iznoc < q3 and iznoc >= q2:
        return "Сильный"
    else:
        return "Экстремальный"

df["Износ"] = df["Износ"].apply(categorize)

print(q0)

1e-05


3. Добавим переменную `"Класс"` для определения класса автомобиля исходя из его цены на рынке. Можно выделить 4 основный класса, а именно:\
Эконом\
Средний\
Премиум\
Люкс

К классу `"Эконом"` будут относиться машины, стоимости которых составляют первые 30% от списка стоимостей всех авто.\
`"Средний"` - это, как правило, самый общирный класс на рынке, поэтому стоимости данных автомобилей будут начинаться от 30% и заканчиваться на 75%.\
`"Премиум"` - здесь начинается роскошь в мире автомобилей, которая заключается в разных дополнительных функциях и качестве автомобиля, соответвенно, цена и от 75% до 95% среди стоимостей всех авто.\
`"Люкс"` - класс, предлагающий абсолютный комфорт, передовые технологии и роскошные материалы. В этот сегмент могут входить люксовые седаны, спортивные автомобили высшего класса и эксклюзивные модели, соответственно, и цена - верхние 5% списка авто.

In [5]:
sp = []
sp1 = []
for index, i in df.iterrows():
    sp.append(i["Цена машины, ₽"])

sp_sorted = np.sort(sp)

p1 = np.percentile(sp_sorted, 30)
p2 = np.percentile(sp_sorted, 75)
p3 = np.percentile(sp_sorted, 95)

In [6]:
sp1 = []
for index, i in df.iterrows():
    price = i["Цена машины, ₽"]
    if price < p1:
        sp1.append("Эконом")
    elif price < p2 and price >= p1:
        sp1.append("Средний")
    elif price < p3 and price >= p2:
        sp1.append("Премиум")
    else:
        sp1.append("Люкс")

df["Класс"] = sp1

4) `ДТП` - машина скорее всего участвовала в дтп. \
\
Для этого необходимо:\
Найти все уникальные модели машин\
Определить квантиль уровня 15% среди цен каждой модели\
\
Какие машины будут сильно ниже средней цены (квантиль уровня 15%) по своей модели при удовлетворительном износе - будут помечаться как возможно участвовавшие в дтп.

In [7]:
unique = df["Модель машины"].unique()
d = {}
for i in unique:
    d[i] = []

In [8]:
for index, i in df.iterrows():
    d[i["Модель машины"]].append(i["Цена машины, ₽"])

In [9]:
for k,v in d.items():
    v_sort = np.sort(v)
    s = np.percentile(v_sort, 15)
    d[k] = s

In [10]:
sp1 = []
for index, i in df.iterrows():
    if i["Цена машины, ₽"] < d[i["Модель машины"]] and i["Износ"] != "Экстремальный":
        sp1.append("True")
    else:
        sp1.append("False")


In [11]:
df["ДТП"] = sp1

In [12]:
df.head(20)

Срок размещения объявления (дней)                   Модель машины  \
0                                   0                   BMW X7 4.4 AT   
1                                   0              BMW 7 серия 3.0 AT   
2                                   0                   BMW X7 3.0 AT   
3                                   0                   BMW XM 4.4 AT   
4                                   5                   BMW X7 3.0 AT   
5                                   1                   BMW X5 3.0 AT   
6                                   0                   BMW X5 3.0 AT   
7                                   3                   BMW X7 3.0 AT   
8                                   2                   BMW X6 3.0 AT   
9                                   1                   BMW X7 4.4 AT   
10                                  1                   BMW X7 3.0 AT   
11                                  4  BMW 2 серия Gran Tourer 2.0 AT   
12                                  4                       BMW iX AT   
13                                  1                   BMW X5 3.0 AT   
14                                  4                   BMW X4 3.0 AT   
15                                  5                   BMW X4 2.0 AT   
16                                  3                   BMW X7 3.0 AT   
17                                  1                   BMW X7 3.0 AT   
18                                  0                   BMW X6 3.0 AT   
19                                  0                   BMW X5 3.0 AT   

    Год выпуска  Пробег, км  Цена машины, ₽   Тип машины    Привод  \
0          2024           1        21900000  внедорожник    полный   
1          2023          50        17000000        седан    полный   
2          2024           1        16900000  внедорожник    полный   
3          2023          15        25490000  внедорожник    полный   
4          2020       65000        10190000  внедорожник    полный   
5          2016      154600         3745000  внедорожник    полный   
6          2024           1        13750000  внедорожник    полный   
7          2019      104000         8050000  внедорожник    полный   
8          2023           0        15900000  внедорожник    полный   
9          2023           0        20700000  внедорожник    полный   
10         2023           0        20585700  внедорожник    полный   
11         2018      102500         2249000      минивэн  передний   
12         2022           0        12950000  внедорожник    полный   
13         2020       68798         8499000  внедорожник    полный   
14         2014      130000         3150000  внедорожник    полный   
15         2019       90057         4100000  внедорожник    полный   
16         2021       13000         9800000  внедорожник    полный   
17         2019       71500         9400000  внедорожник    полный   
18         2024           1        16500000  внедорожник    полный   
19         2019       78000         7350000  внедорожник    полный   

   Тип топлива                                             Ссылка  \
0       бензин  https://www.avito.ru/moskva/avtomobili/bmw_x7_...   
1       дизель  https://www.avito.ru/moskva/avtomobili/bmw_7_s...   
2       дизель  https://www.avito.ru/moskva/avtomobili/bmw_x7_...   
3       гибрид  https://www.avito.ru/moskva/avtomobili/bmw_xm_...   
4       дизель  https://www.avito.ru/moskva/avtomobili/bmw_x7_...   
5       дизель  https://www.avito.ru/moskva/avtomobili/bmw_x5_...   
6       дизель  https://www.avito.ru/moskva/avtomobili/bmw_x5_...   
7       дизель  https://www.avito.ru/moskva/avtomobili/bmw_x7_...   
8       дизель  https://www.avito.ru/moskva/avtomobili/bmw_x6_...   
9       бензин  https://www.avito.ru/moskva/avtomobili/bmw_x7_...   
10      дизель  https://www.avito.ru/moskva/avtomobili/bmw_x7_...   
11      дизель  https://www.avito.ru/moskva/avtomobili/bmw_2_s...   
12     электро  https://www.avito.ru/moskva/avtomobili/bmw_ix_...   
13      дизель  https://www.avito

In [13]:
df.to_csv('Признаки.csv', index=False)